In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from vecstack import stacking

In [39]:
class modeling:
  !pip install catboost
  # train과 test로 나누기
  def train_test_split(data = None):
    train = data[data['date'] <= 201712]
    test = data[data['date'] > 201712]

    X_train, y_train = train.drop('target', axis = 1), train['target']
    X_test, y_test = test.drop('target', axis = 1), test['target']

    return X_train, X_test, y_train, y_test

  # linear regression 돌리기
  def lr():
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    
    return model

  # xgboost 돌리기
  def xgb():
    from xgboost import XGBRegressor
    model = XGBRegressor(random_state = 42)

    return model

  # lightgbm
  def lgb():
    from lightgbm import LGBMRegressor
    model = LGBMRegressor(random_state = 42)

    return model

  # catboost
  def cat():
    from catboost import CatBoostRegressor
    model = CatBoostRegressor(random_state = 42)

    return model

  # ridge
  def ridge():
    from sklearn.linear_model import Ridge
    model = Ridge(random_state = 42)
    
    return model
  
  # lasso
  def lasso():
    from sklearn.linear_model import Lasso
    model = Lasso()

    return model

  # adaboost
  def ada():
    from sklearn.ensemble import AdaBoostRegressor
    model = AdaBoostRegressor(random_state = 42)

    return model

  # randomforest
  def rfg():
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(random_state = 42)

    return model

  def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    RMSE = mean_squared_error(y_test, y_pred)**0.5

    return RMSE


# 모델 전부 한번씩 돌려보기
class class_all_model:
  def models():
    models = {'lr':modeling.lr(), 'lgb':modeling.lgb(), 'xgb':modeling.xgb(), 'lasso':modeling.lasso(),'ridge':modeling.ridge(),'cat':modeling.cat(),'ada':modeling.ada(), 'rfg':modeling.rfg()}
    return models
  
  def run(data = None):
    models = class_all_model.models()

    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    for i, j in models.items():
      model = j
      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)

      print(f'{i} ',modeling.rmse(y_test, y_pred))

# stacking ensemble
class class_stack:
  def stack(data = None):
    !pip install vecstack
    from sklearn.metrics import mean_squared_error
    from vecstack import stacking
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    models = [modeling.lgb(), modeling.cat(), modeling.lr(), modeling.xgb(), modeling.lasso(), modeling.ada(), modeling.rfg(),modeling.ridge()]
    # stacking을 사용하자
    S_train, S_test = stacking(models, X_train, y_train, X_test, regression = True, metric = ['acc'], n_folds = 3)
    model = modeling.lr()
    model.fit(S_train, y_train)
    y_pred = model.predict(S_test)

    return print('stacking :',modeling.rmse(y_test, y_pred))


class class_voting:
  def collect_model():
    models= [('xgb',modeling.xgb()),('lgb',modeling.lgb()),('rfg',modeling.rfg()),('ada',modeling.ada()),('lr',modeling.lr()),('ridge',modeling.ridge()),('lasso',modeling.lasso()),('cat',modeling.cat())]
    return models

  def voting(data = None):
    from sklearn.ensemble import VotingRegressor
    from sklearn.metrics import mean_squared_error
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    model = VotingRegressor(class_voting.collect_model())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return print('voting :',modeling.rmse(y_test, y_pred))

class final_model():
  def run(data = data):
    class_all_model.run(data = data)
    class_stack.stack(data =data)
    class_voting.voting(data =data)

    print('------ 끝 --------------------------------------------------')

In [40]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/sd솔데스크/팀 프로젝트/덕수궁_겨울.csv')
data = data.rename(columns = {'덕수궁':'target'})
data.head()

,date,month,최고 기온(°C),최소 상대습도(%),평균 기온(°C),평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조 시간(hr),0.5m 지중온도(°C),최대 풍속(m/s),합계 일사량(MJ/m2),target,season,겨울철 체감온도,불쾌지수
0,201101,1,-3.406452,3.587097,-7.183871,4.004220,0.747872,1.334152,2.085472,1.541935,1.886092,2.393457,92135,4,23.985270,39.623680
1,201102,2,5.864286,3.496508,1.221429,4.029425,2.370244,1.267953,1.938742,1.025000,1.830866,2.470458,132920,4,33.261087,46.761874
2,201112,12,2.851613,3.486684,-0.870968,3.940391,0.464782,1.285404,1.989199,5.529032,1.815140,2.074713,52140,4,30.955365,44.997064
3,201201,1,1.338710,3.451522,-2.787097,3.909180,0.443617,1.261937,1.966435,1.680645,1.847184,2.147628,40041,4,28.764408,43.375576
4,201202,2,2.968966,3.203587,-2.044828,3.788257,0.125163,1.352096,2.169645,0.968966,1.893320,2.502707,36900,4,29.766218,44.025280


In [41]:
final_model.run(data =data)

lr  59281.91837959518
lgb  43249.31219833734
[15:51:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb  44606.943788771256
lasso  34340.14192086476
ridge  54918.96080057235
Learning rate set to 0.019623
0:	learn: 24868.2653602	total: 2.21ms	remaining: 2.21s
1:	learn: 24713.1377472	total: 3.14ms	remaining: 1.57s
2:	learn: 24659.3238631	total: 4.26ms	remaining: 1.42s
3:	learn: 24471.3789147	total: 5.19ms	remaining: 1.29s
4:	learn: 24338.5413362	total: 6.58ms	remaining: 1.31s
5:	learn: 24164.1314883	total: 7.93ms	remaining: 1.31s
6:	learn: 24046.7960101	total: 9.3ms	remaining: 1.32s
7:	learn: 23921.7705195	total: 10.7ms	remaining: 1.32s
8:	learn: 23762.1001780	total: 12.1ms	remaining: 1.33s
9:	learn: 23625.8637132	total: 13.5ms	remaining: 1.33s
10:	learn: 23519.9337518	total: 14.8ms	remaining: 1.33s
11:	learn: 23384.1745977	total: 16.2ms	remaining: 1.33s
12:	learn: 23261.4166021	total: 17.5ms	remaining: 1.33s
13:	lea

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2236478547.0046463, tolerance: 1312106.386695238
  positive)


112:	learn: 13936.5346853	total: 116ms	remaining: 909ms
113:	learn: 13868.6723259	total: 117ms	remaining: 906ms
114:	learn: 13805.7418492	total: 118ms	remaining: 907ms
115:	learn: 13738.3812096	total: 119ms	remaining: 903ms
116:	learn: 13653.3687882	total: 121ms	remaining: 915ms
117:	learn: 13594.7472280	total: 123ms	remaining: 919ms
118:	learn: 13519.4936020	total: 124ms	remaining: 915ms
119:	learn: 13481.3495123	total: 124ms	remaining: 912ms
120:	learn: 13390.3574306	total: 125ms	remaining: 908ms
121:	learn: 13324.4178176	total: 126ms	remaining: 905ms
122:	learn: 13248.8856153	total: 126ms	remaining: 901ms
123:	learn: 13184.8980041	total: 127ms	remaining: 898ms
124:	learn: 13126.0633523	total: 128ms	remaining: 893ms
125:	learn: 13045.5148129	total: 128ms	remaining: 890ms
126:	learn: 12984.2779620	total: 129ms	remaining: 886ms
127:	learn: 12912.5800258	total: 130ms	remaining: 883ms
128:	learn: 12826.5040132	total: 130ms	remaining: 880ms
129:	learn: 12750.1899763	total: 131ms	remaining

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Learning rate set to 0.018256
0:	learn: 19497.0095474	total: 335us	remaining: 335ms
1:	learn: 19415.3161746	total: 1.05ms	remaining: 526ms
2:	learn: 19355.4919404	total: 1.54ms	remaining: 511ms
3:	learn: 19247.8536741	total: 2.17ms	remaining: 540ms
4:	learn: 19173.4369049	total: 2.8ms	remaining: 557ms
5:	learn: 19084.4441551	total: 3.44ms	remaining: 570ms
6:	learn: 19000.4847852	total: 4.06ms	remaining: 576ms
7:	learn: 18912.8588502	total: 8.52ms	remaining: 1.06s
8:	learn: 18821.1240694	total: 15.3ms	remaining: 1.69s
9:	learn: 18734.1221556	total: 15.9ms	remaining: 1.58s
10:	learn: 18651.6969494	total: 16.5ms	remaining: 1.48s
11:	learn: 18541.8245441	total: 18.3ms	remaining: 1.5s
12:	learn: 18453.9803977	total: 18.9ms	remaining: 1.44s
13:	learn: 18368.8462833	total: 19.5ms	remaining: 1.38s
14:	learn: 18279.5595289	total: 20.1ms	remaining: 1.32s
15:	learn: 18199.9481571	total: 20.5ms	remaining: 1.26s
16:	learn: 18129.0876428	total: 20.9ms	remaining: 1.21s
17:	learn: 18028.5599746	total:

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20717471.46573566, tolerance: 539336.3433214284
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 125476580.3366787, tolerance: 810630.1235214287
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 343793982.58571154, tolerance: 1251055.5243714286
  positive)


stacking : 36920.33591762902
[15:51:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2236478547.0046463, tolerance: 1312106.386695238
  positive)


Learning rate set to 0.019623
0:	learn: 24868.2653602	total: 1.03ms	remaining: 1.03s
1:	learn: 24713.1377472	total: 3.64ms	remaining: 1.81s
2:	learn: 24659.3238631	total: 4.68ms	remaining: 1.55s
3:	learn: 24471.3789147	total: 5.66ms	remaining: 1.41s
4:	learn: 24338.5413362	total: 7.14ms	remaining: 1.42s
5:	learn: 24164.1314883	total: 8.67ms	remaining: 1.44s
6:	learn: 24046.7960101	total: 10.2ms	remaining: 1.45s
7:	learn: 23921.7705195	total: 11.7ms	remaining: 1.45s
8:	learn: 23762.1001780	total: 13.2ms	remaining: 1.46s
9:	learn: 23625.8637132	total: 14.8ms	remaining: 1.46s
10:	learn: 23519.9337518	total: 16.3ms	remaining: 1.46s
11:	learn: 23384.1745977	total: 17.7ms	remaining: 1.46s
12:	learn: 23261.4166021	total: 19.2ms	remaining: 1.46s
13:	learn: 23167.5340193	total: 20.8ms	remaining: 1.47s
14:	learn: 23038.2317131	total: 22.7ms	remaining: 1.49s
15:	learn: 22943.0722494	total: 23.5ms	remaining: 1.44s
16:	learn: 22832.2540841	total: 24.6ms	remaining: 1.42s
17:	learn: 22700.2024378	tot

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   일시             27 non-null     int64  
 1   month          27 non-null     int64  
 2   최고 기온(°C)      27 non-null     float64
 3   최소 상대습도(%)     27 non-null     float64
 4   평균 기온(°C)      27 non-null     float64
 5   평균 상대습도(%)     27 non-null     float64
 6   일강수량(mm)       27 non-null     float64
 7   평균 풍속(m/s)     27 non-null     float64
 8   합계 일조 시간(hr)   27 non-null     float64
 9   0.5m 지중온도(°C)  27 non-null     float64
 10  최대 풍속(m/s)     27 non-null     float64
 11  합계 일사량(MJ/m2)  27 non-null     float64
 12  target         27 non-null     int64  
 13  season         27 non-null     int64  
 14  겨울철 체감온도       27 non-null     float64
 15  불쾌지수           27 non-null     float64
dtypes: float64(12), int64(4)
memory usage: 3.5 KB


In [ ]:
class_stack.stack(data = data)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Learning rate set to 0.018256
0:	learn: 19497.0095474	total: 275us	remaining: 275ms
1:	learn: 19415.3161746	total: 1.01ms	remaining: 506ms
2:	learn: 19355.4919404	total: 1.46ms	remaining: 487ms
3:	learn: 19247.8536741	total: 2.06ms	remaining: 512ms
4:	learn: 19173.4369049	total: 2.73ms	remaining: 543ms
5:	learn: 19084.4441551	total: 3.34ms	remaining: 553ms
6:	learn: 19000.4847852	total: 3.98ms	remaining: 565ms
7:	learn: 18912.8588502	total: 4.57ms	remaining: 567ms
8:	learn: 18821.1240694	total: 5.16ms	remaining: 568ms
9:	learn: 18734.1221556	total: 5.75ms	remaining: 569ms
10:	learn: 18651.6969494	total: 6.35ms	remaining: 571ms
11:	learn: 18541.8245441	total: 6.83ms	remaining: 563ms
12:	learn: 18453.9803977	total: 7.41ms	remaining: 563ms
13:	learn: 18368.8462833	total: 8.01ms	remaining: 564ms
14:	learn: 18279.5595289	total: 8.61ms	remaining: 565ms
15:	learn: 18199.9481571	total: 9.12ms	remaining: 561ms
16:	learn: 18129.0876428	total: 9.73ms	remaining: 562ms
17:	learn: 18028.5599746	tota

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20717471.46573566, tolerance: 539336.3433214284
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 125476580.3366787, tolerance: 810630.1235214287
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 343793982.58571154, tolerance: 1251055.5243714286
  positive)


36920.33591762902

In [ ]:
# stacking ensemble
def hard_stack(data = None):
  X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
  models = [modeling.lgb(), modeling.cat(), modeling.lr(), modeling.xgb(), modeling.lasso(), modeling.ada(), modeling.rfg(),modeling.ridge()]
  # stacking을 사용하자
  S_train, S_test = stacking(models, X_train, y_train, X_test, regression = True, metric = ['acc'], n_folds = 3)

  model = modeling.lr()
  model.fit(S_train, y_train)
  y_pred = model.predict(S_test)
    
  return y_pred, y_test

# voting ensemble
def voting(data =None):
  
hard_stack(data = data)

Learning rate set to 0.018256
0:	learn: 19497.0095474	total: 277us	remaining: 277ms
1:	learn: 19415.3161746	total: 788us	remaining: 394ms
2:	learn: 19355.4919404	total: 2.06ms	remaining: 686ms
3:	learn: 19247.8536741	total: 2.86ms	remaining: 712ms
4:	learn: 19173.4369049	total: 3.74ms	remaining: 745ms
5:	learn: 19084.4441551	total: 4.33ms	remaining: 718ms
6:	learn: 19000.4847852	total: 4.93ms	remaining: 699ms
7:	learn: 18912.8588502	total: 5.5ms	remaining: 682ms
8:	learn: 18821.1240694	total: 6.07ms	remaining: 668ms
9:	learn: 18734.1221556	total: 6.68ms	remaining: 661ms
10:	learn: 18651.6969494	total: 7.45ms	remaining: 670ms
11:	learn: 18541.8245441	total: 7.98ms	remaining: 657ms
12:	learn: 18453.9803977	total: 8.55ms	remaining: 649ms
13:	learn: 18368.8462833	total: 9.14ms	remaining: 644ms
14:	learn: 18279.5595289	total: 9.72ms	remaining: 639ms
15:	learn: 18199.9481571	total: 10.2ms	remaining: 628ms
16:	learn: 18129.0876428	total: 10.8ms	remaining: 624ms
17:	learn: 18028.5599746	total:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


202:	learn: 7237.0019470	total: 124ms	remaining: 488ms
203:	learn: 7203.9020072	total: 125ms	remaining: 488ms
204:	learn: 7166.6530872	total: 126ms	remaining: 487ms
205:	learn: 7135.8164125	total: 126ms	remaining: 487ms
206:	learn: 7098.8878034	total: 128ms	remaining: 491ms
207:	learn: 7065.3592277	total: 129ms	remaining: 491ms
208:	learn: 7033.6994856	total: 130ms	remaining: 491ms
209:	learn: 7003.2415025	total: 130ms	remaining: 490ms
210:	learn: 6971.0372648	total: 131ms	remaining: 491ms
211:	learn: 6931.0186693	total: 134ms	remaining: 499ms
212:	learn: 6901.8077778	total: 135ms	remaining: 498ms
213:	learn: 6863.6074881	total: 136ms	remaining: 498ms
214:	learn: 6831.6290956	total: 136ms	remaining: 497ms
215:	learn: 6801.9179383	total: 137ms	remaining: 496ms
216:	learn: 6768.9641162	total: 137ms	remaining: 495ms
217:	learn: 6739.8988352	total: 138ms	remaining: 494ms
218:	learn: 6709.0342207	total: 138ms	remaining: 494ms
219:	learn: 6677.7682730	total: 139ms	remaining: 493ms
220:	learn

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20717471.46573566, tolerance: 539336.3433214284
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 125476580.3366787, tolerance: 810630.1235214287
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 343793982.58571154, tolerance: 1251055.5243714286
  positive)


(array([81726.14288663, 44734.05504817, 81241.90883076, 67985.52817804,
        96757.22696871, 63319.25936445]), 21     64456
 22     71009
 23    129213
 24    109930
 25    123757
 26    112314
 Name: target, dtype: int64)